# Athena Details

In [ ]:
#Connecting with s3 
import boto3

data_lake_access_role_arn = "arn:aws:iam::511372950887:role/askdb-project-role-llm-bi"
region = "us-east-1"

# get boto3 base session
base_session = boto3.Session()

# get credentials for the data-lake-access role
sts_client = base_session.client("sts")

response = sts_client.assume_role(
            RoleArn=data_lake_access_role_arn,  

            RoleSessionName="example1",)

boto_session_data_lake_access = boto3.Session(
        aws_access_key_id=response["Credentials"]["AccessKeyId"],

        aws_secret_access_key=response["Credentials"]["SecretAccessKey"],

        aws_session_token=response["Credentials"]["SessionToken"],

        region_name=region, )

s3_client = boto_session_data_lake_access.client("s3")

#Retrieve the AWS account number
account_number = sts_client.get_caller_identity().get('Account')
#print("AWS Account Number:", account_number)


In [ ]:
#Connection with Athena using arn role.
import boto3
import time

sts_client = boto3.client('sts')

assumed_role_object = sts_client.assume_role(
    RoleArn='arn:aws:iam::511372950887:role/askdb-project-role-llm-bi',
    RoleSessionName='askdb_account_datalake_access'
)
credentials = assumed_role_object['Credentials']

session = boto3.Session(
    aws_access_key_id=credentials['AccessKeyId'],
    aws_secret_access_key=credentials['SecretAccessKey'],
    aws_session_token=credentials['SessionToken']
)

athena_client=session.client('athena', 'eu-central-1')
glue_client=session.client('glue', 'eu-central-1')

# Testing The Query

In [ ]:
import pandas as pd
import time
import boto3


query = """SELECT DISTINCT(open_delivery_qty_kg), country_final_dest
            FROM llm_dummy_q001
            WHERE country_final_dest IN ('USA','Thailand','Germany');"""




response = athena_client.start_query_execution(
    QueryString=query,
    QueryExecutionContext={
        'Database': 'llm'
    },
    ResultConfiguration={
        'OutputLocation': 's3://askdb-qa-user-data/projects/askdb_dir/athena-results/'
    }
)

query_execution_id = response['QueryExecutionId']

# Wait for query execution to complete
while True:
    query_execution = athena_client.get_query_execution(QueryExecutionId=query_execution_id)
    query_status = query_execution['QueryExecution']['Status']['State']
    
    if query_status in ['SUCCEEDED', 'FAILED', 'CANCELLED']:
        break
    time.sleep(5)

if query_status == 'SUCCEEDED':
    # Get query results
    results = athena_client.get_query_results(QueryExecutionId=query_execution_id)
    #print(results)
    
    # Extract column names
    column_names = [col['Label'] for col in results['ResultSet']['ResultSetMetadata']['ColumnInfo']]
    #print(column_names)
    
    # Extract row values
    row_values = []
    for row in results['ResultSet']['Rows'][1:]:
        row_data = row.get('Data', [])
        row_values.append([cell.get('VarCharValue', '') for cell in row_data])
        #row_values.append([cell['VarCharValue'] for cell in row['Data']])
    
    # Create a DataFrame
    df = pd.DataFrame(row_values, columns=column_names)
    
    # Display the DataFrame
    print(df)
    
    # Save the DataFrame to a CSV file
    #df.to_csv('output.csv', index=False)
else:
    print("Query execution failed")


   open_delivery_qty_kg country_final_dest
0                 2.000           Thailand
1               -40.000            Germany
2                                  Germany
3                 0.000            Germany
4                 1.000                USA
5               -60.000            Germany
6               -28.000            Germany
7                                      USA
8                 0.000                USA
9               -32.000            Germany
10               10.000           Thailand
11                4.000                USA
12              -10.000            Germany
13              -50.000            Germany
14               -8.000            Germany
15            20000.000            Germany
16              -24.000            Germany
17                5.000            Germany
18             -900.000            Germany
19                                Thailand
20                0.000           Thailand
21              -14.000            Germany
22         

In [10]:
df.head()

,open_delivery_qty_kg,country_final_dest
0,2.000,Thailand
1,-40.000,Germany
2,,Germany
3,0.000,Germany
4,1.000,USA


In [14]:
database_response = df.to_string(index=False)

### Part 2: Pass database response to LLM for summarizing the response in natural language

In [21]:
# %pip install --no-build-isolation --force-reinstall \
#     dependencies/awscli-*-py3-none-any.whl \
#     dependencies/boto3-*-py3-none-any.whl \
#     dependencies/botocore-*-py3-none-any.whl

In [27]:
# !pip install --quiet "faiss-cpu>=1.7,<2" "ipywidgets>=7,<8" langchain==0.0.249 "pypdf>=3.8,<4"

In [ ]:
import os
from utils import bedrock, print_ww

os.environ["AWS_DEFAULT_REGION"] = "us-east-1"  
os.environ["BEDROCK_ASSUME_ROLE"] = "arn:aws:iam::126225155318:role/askdb-bedrock-cross-account"  

    
boto3_bedrock = bedrock.get_bedrock_client(assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),                                           region=os.environ.get("AWS_DEFAULT_REGION", None),)

Create new client
  Using region: us-east-1
  Using role: arn:aws:iam::126225155318:role/cov-bedrock-cross-account ... successful!
boto3 Bedrock client successfully created!
bedrock(https://bedrock.us-east-1.amazonaws.com)


In [29]:
from langchain.llms.bedrock import Bedrock
from langchain.chains import ConversationChain

claude_llm = Bedrock(model_id="anthropic.claude-v2", 
                     client=boto3_bedrock, 
                     model_kwargs={"max_tokens_to_sample": 8190, "temperature":0})

conversation = ConversationChain(llm=claude_llm, verbose=False)

In [30]:
final_prompt = f"""Please summarize the following DataFrame: {database_response}"""

In [32]:
print(conversation.predict(input=final_prompt))

 Here is a high-level summary of the key points from the DataFrame:

- Contains open delivery quantities (kg) and destination countries for 25 rows
- 3 countries represented: Thailand, Germany, USA
- Quantities range from -900,000 kg to 20,000 kg  
- Germany has most rows (18) and largest total negative quantity (-1,153,000 kg)
- USA has single largest positive quantity (20,000 kg)
- Totals by country:
  - Thailand: 2,000 kg
  - Germany: -1,153,000 kg
  - USA: 21,774 kg

Let me know if you need any specific additional details summarized!


In [4]:
import streamlit as st


def display_sql_query():
    sql_query = "Select * from employee where id>251;"
    response = f"SQL query: '{sql_query}'"
    return response

def display_summary():
    summary = "There are total 500 employees whose id is greater than 251. Among them 150 are male and 101 are female"
    response = f"summary: '{summary}'"
    return response

def main():
    st.title("Chat with SAP-BW Data using Gen-AI")
    
    
    user_input = st.text_input("Ask your question:")
    
    if st.button("Submit"):
        if user_input:
            
            st.write(f"You asked: '{user_input}'")
            
            sql_response = display_sql_query()
            summary_response = display_summary()

            st.write(sql_response)
            st.write(summary_response)
        else:
            st.warning("Please enter a message.")

if __name__ == "__main__":
    main()


2023-09-18 05:15:22.194 
  command:

    streamlit run /opt/conda/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]


In [7]:
!streamlit run app.py




  You can now view your Streamlit app in your browser.

  Network URL: http://169.255.254.1:8501
  External URL: http://54.154.213.168:8501

^C
  Stopping...
